<a href="https://colab.research.google.com/github/dimitrod/ehu_nlp_dimathina/blob/Evaluation/Evaluation_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/mandarjoshi90/triviaqa.git

Cloning into 'triviaqa'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70 (from 1)
Receiving objects: 100% (70/70), 20.60 KiB | 540.00 KiB/s, done.
Resolving deltas: 100% (28/28), done.


In [4]:
%cd triviaqa

/content/triviaqa


In [5]:
!pip install -r requirements.txt

In [84]:
import subprocess
import os
import json
from tqdm import tqdm
from dummy_model.dummy_model import dummy_model
import argparse

In [53]:
triviaqa_root = "/content/triviaqa"

In [54]:
env = os.environ.copy()
env["PYTHONPATH"] = triviaqa_root

In [18]:
%cd ..

/content


In [66]:
def evaluate_triviaqa(dataset_file, prediction_file):
  script_path = "/content/triviaqa/evaluation/triviaqa_evaluation.py"
  result = subprocess.run([
    "python", script_path,
    "--dataset_file", dataset_file,
    "--prediction_file", prediction_file],
    env=env,
    capture_output=True,
    text=True,
  )
  print(result.stdout)

In [68]:
def get_answers(questions, model):
    answers = {}

    for i in tqdm(range(len(questions)), desc="Retrieving Answers"):
        question = questions[i]
        answers[question[0]] = model.invoke(question[1])
    return answers

In [69]:
def load_questions(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.loads(f.read())

    questions = []

    for i in tqdm(range(len(data["Data"])),desc="Loading Questions"):
        question = data["Data"][i]
        questions.append([question["QuestionId"], question["Question"]])
    return questions

In [70]:
def get_args():
    parser = argparse.ArgumentParser(
        description='Evaluation Chain for TriviaQA {}')
    parser.add_argument('--dataset_file', help='Dataset file')
    parser.add_argument('--model_file', help='Model file')
    args = parser.parse_args()
    return args

In [88]:
if __name__ == "__main__":
    #args = get_args()
    evaluation_path = "/content/datasets/evaluation_data.json"
    predictions_path = "/content/datasets/predictions.json"
    questions = load_questions(evaluation_path)
    model = dummy_model()
    answers = get_answers(questions, model)
    with open(predictions_path, "w", encoding="utf-8") as f:
        f.write(json.dumps(answers))
    evaluate_triviaqa(dataset_file, prediction_file)


Retrieving Answers: 100%|██████████| 7900/7900 [00:00<00:00, 622674.51it/s]


{'exact_match': 100.0, 'f1': 100.0, 'common': 7900, 'denominator': 7900, 'pred_len': 7900, 'gold_len': 7900}

